In [1]:
import pickle
import pandas as pd
import numpy as np
import time


df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Words'] = df['Tweet'].str.count(' ') + 1
df = df[df['Words']>=4]

print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8488073
Russian 2285628


In [2]:
cv = pickle.load(open('models/countVectorizer.pickle', 'rb'))
sc = pickle.load(open('models/scaler.pickle', 'rb'))
lr = pickle.load(open('models/regression.pickle', 'rb'))

In [3]:
df = df[df['Category']==1].copy()

In [4]:
lst = []
for i, key in enumerate(cv.vocabulary_):
    lst.append((key,lr.coef_[0][cv.vocabulary_[key]]))
    
lst.sort(key = lambda x: x[1], reverse=True)
top = []
for a in lst[:100]:
    if a[1] > 0.1:
        top.append(a)
        
top

[('nn _link_ _link_', 0.3797464201532281),
 ('nnp _link_', 0.3596880439375148),
 ('nnp nnp nnp nnp nnp', 0.27202287392636865),
 ('_at_someone_ _at_someone_ _at_someone_', 0.27137328821191053),
 ('nnp nnp nnp nnp', 0.2666274788868331),
 ('_link_ _link_', 0.26506426748952605),
 ('nns _link_ _link_', 0.25811814922820925),
 ('nnp nnp nnp', 0.2578971669917935),
 ('nnp nnp', 0.2535237888547213),
 ('jj nnp', 0.24663079001711583),
 ('jj _link_ _link_', 0.23761461888560031),
 ('nn nn _link_ _link_', 0.21946531849761422),
 ('nnp nnp _link_', 0.1994363537542926),
 ('nnp nnp nnp nnp _link_', 0.1981413163755542),
 ('jj nnp nnp', 0.197819934725734),
 ('jj nnp nnp nnp nnp', 0.19765467811076545),
 ('nnp nnp nnp _link_', 0.19265080977534702),
 ('_at_someone_ nn _at_someone_ nn _at_someone_', 0.19197845005457279),
 ('jj nnp nnp nnp', 0.1897798562005026),
 ('_hashtag_ _hashtag_ _link_', 0.1897109493961316),
 ('in nn _hashtag_', 0.1837502290844699),
 ('_at_someone_ _at_someone_', 0.16713595247883237),
 ('

In [5]:
df['HowRussian'] = lr.predict_proba(sc.transform(cv.transform(df['Processed'])))[:,1]
df = df[df['HowRussian']>=0.9].copy()
df = df.sort_values('HowRussian', ascending=False)
df.head()

,Tweet,Category,Retweet,Processed,Words,HowRussian
796763,#PlayBall! #JackSunday 💻: https://t.co/SxlBNl...,1,False,_hashtag_ _hashtag_ NNP _link_ NNP _link_ NNP ...,10,1.0
874749,Сюрприз для «боевиков» приготовили дагестански...,1,False,JJ NNP NNP NNP NNP _hashtag_ NNP _link_ _hasht...,15,1.0
874197,В САРАТОВСКОЙ ОБЛАСТИ ПЛАНИРУЮТ ОТКРЫТЬ #ПРОИЗ...,1,False,JJ NNP NNP NNP NNP _hashtag_ NNP _link_ _hasht...,14,1.0
887955,В Грузии начата процедура лишения #Саакашвили ...,1,False,JJ NNP NNP NNP NNP _hashtag_ NNP _link_ _hasht...,13,1.0
860293,Насколько прочен «фундамент» власти современно...,1,False,JJ NNP NNP NNP NNP _hashtag_ _link_ NNP _link_,10,1.0


In [6]:
dct = {}
for a in top:
    dct[a[0]] = [a[1], df[df['Processed'].str.contains(a[0],case=False)]]
    


In [7]:
dct[top[1][0]]

[0.3596880439375148,
                                                     Tweet  Category  Retweet  \
 796763  #PlayBall!  #JackSunday 💻: https://t.co/SxlBNl...         1    False   
 874749  Сюрприз для «боевиков» приготовили дагестански...         1    False   
 874197  В САРАТОВСКОЙ ОБЛАСТИ ПЛАНИРУЮТ ОТКРЫТЬ #ПРОИЗ...         1    False   
 887955  В Грузии начата процедура лишения #Саакашвили ...         1    False   
 860293  Насколько прочен «фундамент» власти современно...         1    False   
 889159  Перемога! #Украинцы научились сбивать #РФ #ист...         1    False   
 874088  БОЛЕЕМ ЗА НАШИХ!���  #Россия - #Норвегия.   Он...         1    False   
 887986  Дочь президента Украины снялась в #РФ сериале....         1    False   
 873631  Первому выходу человека в открытый #космос пос...         1    False   
 890237  Уроженцы Запорожья расстреляли пограничника в ...         1    False   
 873187  Украинская Госпогранслужа: оружия в гумконвоях...         1    False   
 891042